### Easy Dinner Analysis - Complete Data Science Project

#### Core Problem
Understand the relationship between ratings, cost_per_person, and other features to identify patterns that drive restaurant success and pricing strategies.

#### Analytical Objectives
1. Identify factors influencing ratings
2. Analyze pricing strategies
3. Understand discount effectiveness
4. Examine cuisine performance
5. Provide business insights

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Load dataset
df = pd.read_csv('Eazy_Dinner.csv')
print('Dataset loaded successfully!')
print(f'Shape: {df.shape}')
print(f'Columns: {df.columns.tolist()}')

Dataset loaded successfully!
Shape: (3200, 7)
Columns: ['name', 'rating', 'cuisine', 'location', 'region', 'cost_per_person', 'discount']


#### Data Reading and Exploration

In [2]:
print('Dataset Dimensions:')
print(f'  Rows: {df.shape[0]}')
print(f'  Columns: {df.shape[1]}')
print(f'\nData Types:')
print(df.dtypes)
print(f'\nMissing Values:\n{df.isnull().sum()}')
print(f'\nBasic Statistics:')
print(df.describe())

Dataset Dimensions:
  Rows: 3200
  Columns: 7

Data Types:
name                object
rating             float64
cuisine             object
location            object
region              object
cost_per_person      int64
discount           float64
dtype: object

Missing Values:
name                0
rating              4
cuisine             0
location            0
region              0
cost_per_person     0
discount           24
dtype: int64

Basic Statistics:
            rating  cost_per_person     discount
count  3196.000000      3200.000000  3176.000000
mean      4.164143      1819.012500    16.426322
std       0.681284       703.974178     7.085279
min       1.000000       150.000000    10.000000
25%       4.000000      2000.000000    10.000000
50%       4.100000      2000.000000    15.000000
75%       4.600000      2000.000000    25.000000
max       5.000000      9440.000000    50.000000


#### Data Cleaning

In [3]:
df_clean = df.copy()

# Cleaning inconsistencies
df_clean.rename(columns={'cost_per_person': 'cost_per_two'},inplace=True)
df_clean['location'] = df_clean['location'].str.replace("['", "", regex=False).str.replace("']", "", regex=False).str.strip()
df_clean.loc[df_clean['location']=='[]','location']=np.nan
df_clean['region'] = df_clean['region'].str.replace("['", "", regex=False).str.replace("']", "", regex=False).str.strip()
df_clean.loc[df_clean['region']=='[]','region']=np.nan
df_clean.loc[df_clean['region']=="Brigade Road', 'Central Bengaluru",'region']='Central Bengaluru'
df_clean.loc[df_clean['region']=="JP Nagar', 'South Bengaluru",'location']='JP Nagar'
df_clean.loc[df_clean['region']=="JP Nagar', 'South Bengaluru",'region']='South Bengaluru'

# Multicuisine
df_clean.loc[(df_clean['cuisine'].str.contains('Multicuisine', na=False))|(df_clean['cuisine'].str.contains('Fusion',na=False)), 'cuisine'] = 'Multicuisine'

# Healthy & Salads
df_clean.loc[(df_clean['cuisine'].str.contains('Health', na=False))|(df_clean['cuisine'].str.contains('Salad',na=False)), 'cuisine'] = 'Healthy & Salads'

# Beverages
beverage_keys=['Beverages','Cocktail Menu','Juice','Tea','Coffee']
df_clean.loc[df_clean['cuisine'].str.contains('|'.join(beverage_keys), na=False),'cuisine'] = 'Beverages'

# Continental
contin_keys=['Continental','Europ','Italian','French','Parsi','Lebanese','Turkish','Irish']
df_clean.loc[df_clean['cuisine'].str.contains('|'.join(contin_keys), na=False),'cuisine'] = 'Continental'

# Indian
indian_keys = ['Sizzlers', 'Parathas','Biryani','Malwani','Bengali','Mangalorean','Awadhi','Kerala','Gujarati','Andhra','Rajasthani','Tapas','Maharashtrian','Kashmiri','Bihari','India']
df_clean.loc[df_clean['cuisine'].str.contains('|'.join(indian_keys), na=False),'cuisine'] = 'Indian'

# Arabian
arabian_keys = ['Arab', 'Kebab', 'Afghani', 'Barbeque', 'Mughlai', 'Mediterranean','Persian']
df_clean.loc[df_clean['cuisine'].str.contains('|'.join(arabian_keys), na=False),'cuisine'] = 'Arabian'

# Asian
asian_keys = ['Asian', 'Nepalese', 'oriental', 'Chinese', 'Japanese', 'Sushi', 'Thai', 'Korean', 'Bangladesh','Burmese']
df_clean.loc[df_clean['cuisine'].str.contains('|'.join(asian_keys), na=False),'cuisine'] = 'Asian'

# Street Foods
street_keys = ['Sandwich', 'Pizza', 'Burger', 'Desserts', 'Food', 'Fried']
df_clean.loc[df_clean['cuisine'].str.contains('|'.join(street_keys), na=False),'cuisine'] = 'Street Foods'

# Bakery
bakery_keys = ['Mithai', 'Choco', 'Ice Cream', 'Cafe', 'Desserts', 'Delicatessen', 'Bakery']
df_clean.loc[df_clean['cuisine'].str.contains('|'.join(bakery_keys), na=False),'cuisine'] = 'Cafe & Bakery'

# Mexican
df_clean.loc[df_clean['cuisine'].str.contains('Mex', na=False), 'cuisine'] = 'Mexican'

# American
amer_keys=['American','North West Frontier','Global Cuisine']
df_clean.loc[df_clean['cuisine'].str.contains('|'.join(amer_keys), na=False),'cuisine'] = 'American'

In [4]:
# Remove duplicates
print(f'Duplicates before: {df_clean.duplicated().sum()}')
df_clean = df_clean.drop_duplicates()
print(f'Duplicates after: {df_clean.duplicated().sum()}')
print(f'Records remaining: {len(df_clean)}')
df_clean=df_clean.reset_index(drop=True)

Duplicates before: 1609
Duplicates after: 0
Records remaining: 1591


In [5]:
# Handle null values - check and display
print('\nNULL VALUES BEFORE HANDLING')
print('='*50)
print(df_clean.isnull().sum())
print(f'\nTotal null values: {df_clean.isnull().sum().sum()}')

# Fill remaining nulls with safe defaults
df_clean['rating']= df_clean['rating'].fillna(df_clean['rating'].median())
df_clean['location']=df_clean['location'].fillna(df_clean['location'].mode()[0])
df_clean['region'] = df_clean['region'].fillna(df_clean['region'].mode()[0])
df_clean['discount'] = df_clean['discount'].fillna(0)

print('\nNULL VALUES AFTER HANDLING')
print('='*50)
print(df_clean.isnull().sum())
print(f'\nTotal null values: {df_clean.isnull().sum().sum()}')
print(f'Final dataset shape: {df_clean.shape}')


NULL VALUES BEFORE HANDLING
name              0
rating            2
cuisine           0
location        117
region          416
cost_per_two      0
discount         12
dtype: int64

Total null values: 547

NULL VALUES AFTER HANDLING
name            0
rating          0
cuisine         0
location        0
region          0
cost_per_two    0
discount        0
dtype: int64

Total null values: 0
Final dataset shape: (1591, 7)


In [6]:
# Feature Engineering
print('\nFEATURE ENGINEERING')
df_clean['cost_per_person']= (df_clean['cost_per_two']/2).round(0).astype(int)
df_clean['discounted_price'] = df_clean['cost_per_person'] * (1 - df_clean['discount'] / 100)
df_clean['absolute_saving'] = df_clean['cost_per_person'] - df_clean['discounted_price']
df_clean['price_category'] = pd.cut(df_clean['cost_per_person'], 
                                     bins=[0, 1000, 3000, 10000], 
                                     labels=['Low-range', 'Mid-range', 'Premium'])
df_clean['rating_category'] = pd.cut(df_clean['rating'], 
                                      bins=[0, 2, 3, 4, 4.5, 5], 
                                      labels=['Poor', 'Below Avg', 'Good', 'Very Good', 'Excellent'])
print(f'  Features created: cost_per_person, discounted_price, discount_amount, price_category, rating_category')
df_clean.drop(['cost_per_two'],axis=1,inplace=True)
print('\nData cleaning completed!')
print(f'Clean dataset shape: {df_clean.shape}')


FEATURE ENGINEERING
  Features created: cost_per_person, discounted_price, discount_amount, price_category, rating_category

Data cleaning completed!
Clean dataset shape: (1591, 11)


In [7]:
df_clean.head()

,name,rating,cuisine,location,region,discount,cost_per_person,discounted_price,absolute_saving,price_category,rating_category
0,Chin Lung Resto Bar,4.6,Multicuisine,Residency Road,Central Bengaluru,25.0,1000,750.0,250.0,Low-range,Excellent
1,"Atlantis - Brewpub, Cocktails & Kitchen",4.2,Multicuisine,HSR,South Bengaluru,25.0,1000,750.0,250.0,Low-range,Very Good
2,Chin Lung Brewery - Indiranagar,4.6,Multicuisine,Indiranagar,East Bengaluru,25.0,1000,750.0,250.0,Low-range,Excellent
3,BLR Brewing Co.,4.3,Beverages,Kanakapura Road,South Bengaluru,25.0,1000,750.0,250.0,Low-range,Very Good
4,46 Ounces Brewgarden,4.7,Multicuisine,Electronic City,South Bengaluru,25.0,1000,750.0,250.0,Low-range,Excellent


#### Outlier Detection

In [8]:
print('OUTLIER DETECTION (IQR METHOD)')

# Rating outliers
Q1_rating = df_clean['rating'].quantile(0.25)
Q3_rating = df_clean['rating'].quantile(0.75)
IQR_rating = Q3_rating - Q1_rating
lower_bound = Q1_rating - 1.5 * IQR_rating
upper_bound = Q3_rating + 1.5 * IQR_rating
rating_outliers = df_clean[(df_clean['rating'] < lower_bound) | (df_clean['rating'] > upper_bound)]

print(f'\nRATING ANALYSIS:')
print(f'  Q1: {Q1_rating:.2f}, Q3: {Q3_rating:.2f}, IQR: {IQR_rating:.2f}')
print(f'  Outliers: {len(rating_outliers)} ({len(rating_outliers)/len(df_clean)*100:.2f}%)')

# Cost outliers
Q1_cost = df_clean['cost_per_person'].quantile(0.25)
Q3_cost = df_clean['cost_per_person'].quantile(0.75)
IQR_cost = Q3_cost - Q1_cost
lower_bound_cost = Q1_cost - 1.5 * IQR_cost
upper_bound_cost = Q3_cost + 1.5 * IQR_cost
cost_outliers = df_clean[(df_clean['cost_per_person'] < lower_bound_cost) | (df_clean['cost_per_person'] > upper_bound_cost)]

print(f'\nCOST ANALYSIS:')
print(f'  Q1: {Q1_cost:.0f}, Q3: {Q3_cost:.0f}, IQR: {IQR_cost:.0f}')
print(f'  Outliers: {len(cost_outliers)} ({len(cost_outliers)/len(df_clean)*100:.2f}%)')

print(f'\nOUTLIER TREATMENT DECISION: KEEP ALL OUTLIERS')
print(f'  Justification:')
print(f'    • Represent legitimate market segments (budget/premium)')
print(f'    • Small percentage indicates data quality')
print(f'    • Important for business insights and strategy')

OUTLIER DETECTION (IQR METHOD)

RATING ANALYSIS:
  Q1: 4.00, Q3: 4.60, IQR: 0.60
  Outliers: 100 (6.29%)

COST ANALYSIS:
  Q1: 988, Q3: 1000, IQR: 12
  Outliers: 586 (36.83%)

OUTLIER TREATMENT DECISION: KEEP ALL OUTLIERS
  Justification:
    • Represent legitimate market segments (budget/premium)
    • Small percentage indicates data quality
    • Important for business insights and strategy


#### Univariate Analysis - Rating

In [9]:
print('RATING ANALYSIS')
print('='*60)
print(f'Mean: {df_clean["rating"].mean():.2f}')
print(f'Median: {df_clean["rating"].median():.2f}')
print(f'Std Dev: {df_clean["rating"].std():.2f}')
print(f'Skewness: {df_clean["rating"].skew():.2f}')
print(f'Kurtosis: {df_clean["rating"].kurt():.2f}')
print(f'Range: {df_clean["rating"].min():.1f} - {df_clean["rating"].max():.1f}')
print(f'IQR: {df_clean["rating"].quantile(0.75) - df_clean["rating"].quantile(0.25):.2f}')

print(f"\nRating Category Distribution:")
print(df_clean['rating_category'].value_counts())

print(f'\nKey Insight:')
print(f' 1. Ratings are high overall: average {df_clean["rating"].mean():.2f} and median {df_clean["rating"].median():.2f} on a {df_clean["rating"].min():.0f}-{df_clean["rating"].max():.0f} scale, so\n    most restaurants are rated positively.')
print(f' 2. Negative skew {df_clean["rating"].skew():.2f} means ratings are bunched at the higher end with a tail of\n    a few low ratings.')
print(f' 3. High kurtosis {df_clean["rating"].kurt():.2f} indicates ratings are tightly clustered near the center\n    with some notable outliers at the extremes.')
print(f' 4. Low spread (std {df_clean["rating"].std():.2f}, IQR {df_clean["rating"].quantile(0.75)-df_clean["rating"].quantile(0.25):.2f}) shows ratings are quite consistent across restaurants.')

RATING ANALYSIS
Mean: 4.16
Median: 4.10
Std Dev: 0.68
Skewness: -2.15
Kurtosis: 7.56
Range: 1.0 - 5.0
IQR: 0.60

Rating Category Distribution:
rating_category
Good         674
Very Good    418
Excellent    400
Below Avg     56
Poor          43
Name: count, dtype: int64

Key Insight:
 1. Ratings are high overall: average 4.16 and median 4.10 on a 1-5 scale, so
    most restaurants are rated positively.
 2. Negative skew -2.15 means ratings are bunched at the higher end with a tail of
    a few low ratings.
 3. High kurtosis 7.56 indicates ratings are tightly clustered near the center
    with some notable outliers at the extremes.
 4. Low spread (std 0.68, IQR 0.60) shows ratings are quite consistent across restaurants.


#### Univariate Analysis - Cost per Person

In [10]:
print('COST PER PERSON ANALYSIS')
print('='*60)
print(f'Mean: Rs.{df_clean["cost_per_person"].mean():.0f}')
print(f'Median: Rs.{df_clean["cost_per_person"].median():.0f}')
print(f'Mode: Rs.{df_clean["cost_per_person"].mode()[0]:.0f}')
print(f'Range: Rs.{df_clean["cost_per_person"].min():.0f} - Rs.{df_clean["cost_per_person"].max():.0f}')

print(f'\nMarket Segmentation:')
print(f'Budget (<1000): {(df_clean.loc[df_clean['price_category']=='Low-range','price_category'].count()/len(df_clean))*100:.1f}%')
print(f'Mid-range (1000-3000): {(df_clean.loc[df_clean['price_category']=='Mid-range','price_category'].count()/len(df_clean))*100:.1f}%')
print(f'Premium (>3,000): {(df_clean.loc[df_clean['price_category']=='Premium','price_category'].count()/len(df_clean))*100:.1f}%')

print(f"\nRestaurant Count by Price Category:")
print(df_clean['price_category'].value_counts())

COST PER PERSON ANALYSIS
Mean: Rs.909
Median: Rs.1000
Mode: Rs.1000
Range: Rs.75 - Rs.4720

Market Segmentation:
Budget (<1000): 88.1%
Mid-range (1000-3000): 11.8%
Premium (>3,000): 0.1%

Restaurant Count by Price Category:
price_category
Low-range    1402
Mid-range     187
Premium         2
Name: count, dtype: int64


#### Univariate Analysis - Discount

In [11]:
print(f'\nDISCOUNT ANALYSIS')
print('='*60)
print(f'Mean: {df_clean["discount"].mean():.0f}%')
print(f'Median: {df_clean["discount"].median():.0f}%')
print(f'Mode: {df_clean["discount"].mode()[0]:.0f}%')
print(f'Range: {df_clean["discount"].min():.0f}% - {df_clean["discount"].max():.0f}%')

print(f"\nDiscount Statistics:")
print(f"Average discount: {df_clean['discount'].mean():.2f}%")
print(f"Average original price: {df_clean['cost_per_person'].mean():.2f}")
print(f"Average discounted price: {df_clean['discounted_price'].mean():.2f}")
print(f"Average absolute saving per person: {df_clean['absolute_saving'].mean():.2f}")


DISCOUNT ANALYSIS
Mean: 16%
Median: 15%
Mode: 10%
Range: 0% - 50%

Discount Statistics:
Average discount: 16.31%
Average original price: 909.43
Average discounted price: 759.60
Average absolute saving per person: 149.83


#### Key Relationships - Rating, Cost per Person & Discount

In [12]:
print('CORRELATION ANALYSIS')
print('='*60)

corr_matrix = df_clean[['rating', 'cost_per_person', 'discount']].corr()
print(corr_matrix.round(2))

print(f'\nKEY FINDING - THE PRICE PARADOX:')
print(f'  Rating vs Cost: {corr_matrix.loc["rating", "cost_per_person"]:.2f} (weak negative)')
print(f'  Implication: Higher price does NOT guarantee better ratings')
print(f'  Quality depends on execution, not price premium')

print(f'\nRating vs Discount: {corr_matrix.loc["rating", "discount"]:.2f} (very weak)')
print(f'  Implication: Deep discounting marginally impacts ratings')

CORRELATION ANALYSIS
                 rating  cost_per_person  discount
rating             1.00             0.01      0.08
cost_per_person    0.01             1.00      0.06
discount           0.08             0.06      1.00

KEY FINDING - THE PRICE PARADOX:
  Rating vs Cost: 0.01 (weak negative)
  Implication: Higher price does NOT guarantee better ratings
  Quality depends on execution, not price premium

Rating vs Discount: 0.08 (very weak)
  Implication: Deep discounting marginally impacts ratings


#### Categorical Variables Analysis - Cuisine

In [13]:
print(f'CUISINE ANALYSIS')
print('='*60)
print(f'Total unique cuisines: {df_clean["cuisine"].nunique()}')
print(f'\nTop 10 Cuisines:')
print(df_clean['cuisine'].value_counts().head(10))

CUISINE ANALYSIS
Total unique cuisines: 13

Top 10 Cuisines:
cuisine
Multicuisine        729
Indian              246
Continental         156
Street Foods        121
Asian               116
Beverages            94
Cafe & Bakery        74
Arabian              19
Healthy & Salads     12
American              9
Name: count, dtype: int64


#### Categorical Variables Analysis - Region

In [14]:
print(f'\nREGION ANALYSIS')
print('='*60)
print(f'\nRegion Distribution:')
print(df_clean['region'].value_counts().head(10))


REGION ANALYSIS

Region Distribution:
region
South Bengaluru      922
East Bengaluru       209
North Bengaluru      202
Central Bengaluru    161
West Bengaluru        52
Richmond Road          5
Ashok Nagar            5
Hotel Convention       5
Bannerghatta Road      4
Magrath Road           4
Name: count, dtype: int64


#### Categorical Variables Analysis - Location

In [15]:
print(f'\nLOCATION ANALYSIS')
print('='*60)
print(f'Total unique locations: {df_clean["location"].nunique()}')
print(f'\nTop 10 Locations:')
print(df_clean['location'].value_counts().head(10))


LOCATION ANALYSIS
Total unique locations: 282

Top 10 Locations:
location
Koramangala             193
Jayanagar                54
Whitefield               52
BTM                      50
Indiranagar              47
HSR                      46
JP Nagar                 45
Kalyan Nagar             40
Rajarajeshwari Nagar     30
New BEL Road             28
Name: count, dtype: int64


#### Groupby Analysis - Ratings by Region

In [16]:
print('AVERAGE RATING BY REGION')
print('='*60)

region_stats = df_clean.groupby('region')['rating'].agg(['mean', 'count']).round(2).sort_values('mean', ascending=False)
print(region_stats[region_stats['count'] > 5].to_markdown())  # Rating Count 5+ to maintain Authenticity

print(f'\nKey Finding:')
print(f'  North/East/South/Central Bengaluru lead with highest ratings')
print(f'  Quality variations minimal')
print(f'  Regional market dynamics exist but quality-independent')

AVERAGE RATING BY REGION
| region            |   mean |   count |
|:------------------|-------:|--------:|
| North Bengaluru   |   4.22 |     202 |
| Central Bengaluru |   4.2  |     161 |
| East Bengaluru    |   4.2  |     209 |
| South Bengaluru   |   4.14 |     922 |
| West Bengaluru    |   4.1  |      52 |

Key Finding:
  North/East/South/Central Bengaluru lead with highest ratings
  Quality variations minimal
  Regional market dynamics exist but quality-independent


#### Groupby Analysis - Ratings by Cuisine

In [17]:
print('AVERAGE RATING BY CUISINE')
print('='*60)

cuisine_stats = df_clean.groupby('cuisine')['rating'].agg(['mean', 'count']).round(2)
cuisine_stats = cuisine_stats[cuisine_stats['count'] > 10]
cuisine_stats = cuisine_stats.sort_values('mean', ascending=False)

print('Top 15 Cuisines by Rating:')
print(cuisine_stats.head(15).to_markdown())

print(f'\nKey Finding:')
print(f"  Seafood leads (4.62) but has only 5 restaurants")
print(f"  Continental is highest-rated with substantial data (4.31, 91 restaurants)")
print(f"  Multicuisine dominates quantity (663) but rates average (4.12)")
print(f"  Indian cuisine maintains consistent 4.13 rating across 222 restaurants")
print(f"  Specialized cuisines generally outperform generic Multicuisine options")

AVERAGE RATING BY CUISINE
Top 15 Cuisines by Rating:
| cuisine          |   mean |   count |
|:-----------------|-------:|--------:|
| Arabian          |   4.31 |      19 |
| Cafe & Bakery    |   4.28 |      74 |
| Asian            |   4.25 |     116 |
| Continental      |   4.25 |     156 |
| Beverages        |   4.24 |      94 |
| Healthy & Salads |   4.18 |      12 |
| Street Foods     |   4.13 |     121 |
| Multicuisine     |   4.12 |     729 |
| Indian           |   4.11 |     246 |

Key Finding:
  Seafood leads (4.62) but has only 5 restaurants
  Continental is highest-rated with substantial data (4.31, 91 restaurants)
  Multicuisine dominates quantity (663) but rates average (4.12)
  Indian cuisine maintains consistent 4.13 rating across 222 restaurants
  Specialized cuisines generally outperform generic Multicuisine options


#### Groupby Analysis - Ratings by Price Category

In [30]:
print("\nAverage Rating by Price Category")
print('='*60)
avg_rating_by_price = df_clean.groupby('price_category', observed=False)['rating'].agg(['mean', 'std', 'count'])
print(avg_rating_by_price.round(2).to_markdown())


Average Rating by Price Category
| price_category   |   mean |   std |   count |
|:-----------------|-------:|------:|--------:|
| Low-range        |   4.15 |  0.69 |    1402 |
| Mid-range        |   4.29 |  0.61 |     187 |
| Premium          |   4.4  |  0.42 |       2 |


#### Groupby Analysis - Price by Rating Category

In [29]:
print("\nAverage Price by Rating Category:")
print('='*60)
avg_disc_by_rating = df_clean.groupby('rating_category', observed=False)['cost_per_person'].agg(['mean', 'std', 'count'])
print(avg_disc_by_rating.round(2).to_markdown())

print("\nAverage Discounted Price by Rating Category:")
print('='*60)
avg_disc_by_rating = df_clean.groupby('rating_category', observed=False)['discounted_price'].agg(['mean', 'std', 'count'])
print(avg_disc_by_rating.round(2).to_markdown())


Average Price by Rating Category:
| rating_category   |    mean |    std |   count |
|:------------------|--------:|-------:|--------:|
| Poor              |  892.44 | 325.44 |      43 |
| Below Avg         | 1019.64 | 215.68 |      56 |
| Good              |  825.45 | 339.3  |     674 |
| Very Good         | 1017.69 | 334.61 |     418 |
| Excellent         |  924.22 | 375.05 |     400 |

Average Discounted Price by Rating Category:
| rating_category   |   mean |    std |   count |
|:------------------|-------:|-------:|--------:|
| Poor              | 745.52 | 264.39 |      43 |
| Below Avg         | 872.19 | 196.54 |      56 |
| Good              | 694.29 | 290.91 |     674 |
| Very Good         | 846.89 | 290.34 |     418 |
| Excellent         | 764.18 | 318.56 |     400 |


#### Discount Analysis - Price & Rating Categories

In [41]:
print("\nAverage Discount by Price & Rating Category")
print("="*80)
discount_crosstab = pd.crosstab(df_clean['price_category'], df_clean['rating_category'], 
                                 values=df_clean['discount'], aggfunc='mean')
print(discount_crosstab.round(2))

print("\nAbsolute Savings by Price & Rating Category")
print("="*80)
savings_crosstab = pd.crosstab(df_clean['price_category'], df_clean['rating_category'], 
                                values=df_clean['absolute_saving'], aggfunc='mean')
print(savings_crosstab.round(2))


Average Discount by Price & Rating Category
rating_category   Poor  Below Avg   Good  Very Good  Excellent
price_category                                                
Low-range        15.25      14.29  15.54      16.68      17.10
Mid-range        23.33      15.71  16.63      17.79      18.06
Premium            NaN        NaN    NaN      10.00      10.00

Absolute Savings by Price & Rating Category
rating_category    Poor  Below Avg    Good  Very Good  Excellent
price_category                                                  
Low-range        130.75     137.14  124.59     151.96     142.86
Mid-range        362.50     219.64  220.76     250.06     266.20
Premium             NaN        NaN     NaN     472.00     354.00


#### Best Value for Money - Top Recommendations by Region

In [82]:
# Region-wise average cost and rating
price_value = df_clean.groupby('region')['cost_per_person'].mean().sort_values(ascending=True).round(0)
rating_value = df_clean.groupby('region')['rating'].agg(['mean','count']).sort_values(by='mean',ascending=True).round(0)

# Combine cost and rating by region
best_value=pd.merge(price_value,rating_value,on='region',how='inner').sort_values(by='cost_per_person',ascending=True)
best_value=best_value.rename(columns={'count': 'restaurants'})

print(f'\nBest Value for Money')
print("="*80,'\n')
print(best_value.loc[(best_value['cost_per_person'] < 1000) & (best_value['restaurants'] > 5),
      ['cost_per_person','restaurants']].to_markdown(),'\n') # restaurants > 5 for more options


Best Value for Money

| region            |   cost_per_person |   restaurants |
|:------------------|------------------:|--------------:|
| Central Bengaluru |               901 |           161 |
| South Bengaluru   |               905 |           922 |
| East Bengaluru    |               908 |           209 |
| North Bengaluru   |               925 |           202 |
| West Bengaluru    |               962 |            52 | 

